In [2]:
# from transformers import pipeline

# # Replace with your repository name
# repo_name = "uday91/llama-2-7b-customer-support-llm"

# # Create a pipeline for text generation
# generator = pipeline("text-generation", model=repo_name, token="")

# # Get inference from your model
# output = generator("help to see when my purchase is going to arrive?", max_length=100, num_return_sequences=1)
# print(output)

In [8]:
#!pip  -q install  accelerate

In [9]:
#!pip  -q install  peft

In [10]:
#!pip install accelerate
#!pip install transformers

In [11]:
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
%pip install accelerate peft bitsandbytes transformers trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch


In [4]:
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token('')

from huggingface_hub.hf_api import HfFolder

# Check if the token was saved
token = HfFolder.get_token()
if token:
    print("Token saved successfully!")
else:
    print("Token not saved.")

Token saved successfully!


In [5]:
base_model = "meta-llama/Meta-Llama-3-8B"


In [7]:
#base_model
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
!huggingface-cli login --token $secret_value_0


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
 base_model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,

        torch_dtype=torch.float16,
        device_map="auto",
    )

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [9]:
base_model_tok = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(base_model_tok)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [11]:
peft_model = "uday91/Meta-Llama-3-8B-customer-support-chat"

In [12]:
print("Loading adapter:")
model = PeftModel.from_pretrained(base_model, peft_model, device_map="auto")



Loading adapter:


adapter_config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

In [13]:
print("Merge base model and adapter")
model = model.merge_and_unload()

Merge base model and adapter


In [14]:
print("Saving model and tokenizer")
model.save_pretrained("Meta-Llama-3-8B-customer-support-chat-merged")
tokenizer.save_pretrained("Meta-Llama-3-8B-customer-support-chat-merged")

Saving model and tokenizer


('Meta-Llama-3-8B-customer-support-chat-merged/tokenizer_config.json',
 'Meta-Llama-3-8B-customer-support-chat-merged/special_tokens_map.json',
 'Meta-Llama-3-8B-customer-support-chat-merged/tokenizer.json')

In [15]:
model.push_to_hub("uday91/Meta-Llama-3-8B-customer-support-finetune", check_pr=True)

tokenizer.push_to_hub("uday91/Meta-Llama-3-8B-customer-support-finetune",check_pr=True)

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/uday91/Meta-Llama-3-8B-customer-support-finetune/commit/9200995505817dfc634c28944cfbe128455005f6', commit_message='Upload tokenizer', commit_description='', oid='9200995505817dfc634c28944cfbe128455005f6', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
#!pip install accelerate

In [4]:
#pip install -i https://pypi.org/simple/ bitsandbytes


In [ ]:
from huggingface_hub.hf_api import HfFolder
HfFolder.save_token('')

from huggingface_hub.hf_api import HfFolder

# Check if the token was saved
token = HfFolder.get_token()
if token:
    print("Token saved successfully!")
else:
    print("Token not saved.")

Token saved successfully!


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [5]:

# # Local directory for Meta's Llama 2 model
# model_id = "meta-llama/Llama-2-7b-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id,load_in_4bit=True,bnb_config)

# model = PeftModel.from_pretrained(model, "uday91/llama-2-7b-customer-support-chat")


In [6]:
# from transformers import AutoModelForCausalLM, AutoTokenizer,AutoConfig
# from transformers import pipeline
# from peft import PeftModel, PeftConfig



In [7]:
# peft_model_id = "uday91/llama-2-7b-customer-support-chat"

# config = AutoConfig.from_pretrained(peft_model_id)

# model = AutoModelForCausalLM.from_pretrained(peft_model_id,config)

In [8]:

# pipe = pipeline("text-generation",
#                 model=model,
#                 tokenizer=tokenizer,
#                 max_new_tokens=200,
# )

# prompt = "Sort an array:"

# print(pipe(prompt)[0]['generated_text'])

In [9]:

# # Save fine-tuned model
# # trainer_filepath= f"trainer/llama7b/{train_util.get_time()}"
# # trainer.model.save_pretrained(trainer_filepath)
# model_name = "meta-llama/Llama-2-7b-hf"

# # reload base model
# base_model= AutoModelForCausalLM.from_pretrained(model_name, token= "")

# # merge base model and fine-tuned model
# merged_model= PeftModel.from_pretrained(base_model, trainer_filepath)
# merged_model= merged_model.merge_and_unload()

# # save merged model
# merged_model_path= f"model/llama7b/merged_{train_util.get_time()}"
# merged_model.save_pretrained(merged_model_path)